In [1]:
import pandas as pd

file_path = '各行业政策总结.xlsx'

# 1. 读取所有 sheet（不使用默认标题行）
all_sheets = pd.read_excel(
    file_path,
    sheet_name=None,      # 读取所有工作表
    header=None           # 明确指定：没有标题行（数据从第0行开始）
)

# 2. 为每个 sheet 手动设置列名
column_names = ['标题', '制定机关', '公布日期']

# 遍历字典，给每个 DataFrame 赋予新列名
for sheet_name in all_sheets:
    df = all_sheets[sheet_name]
    # 确保列数匹配（可选：防止出错）
    if df.shape[1] == len(column_names):
        df.columns = column_names
    else:
        print(f"警告：工作表 '{sheet_name}' 的列数 ({df.shape[1]}) 与标题数量 ({len(column_names)}) 不匹配！")
# all_sheets 是一个 dict，键是 sheet 名称，值是对应的 DataFrame
for sheet_name, df in all_sheets.items():
    print(f"Sheet: {sheet_name}")

Sheet: 汽车（国家）
Sheet: 汽车（地方）
Sheet: 家用电器（国家）
Sheet: 家用电器（地方）
Sheet: 餐饮（国家）
Sheet: 餐饮（地方）
Sheet: 旅游（国家）
Sheet: 旅游（地方法规）
Sheet: 旅游（地方通知）
Sheet: 动漫游戏（国家）
Sheet: 动漫游戏（地方）
Sheet: 文娱（国家）
Sheet: 文娱（地方）


In [2]:
all_sheets["餐饮（地方）"]

,标题,制定机关,公布日期
0,2025年贵州省建设项目使用林地审核审批暨林木采伐管理培训班住宿、餐饮、会场服务询价比选公告,贵州省林业局,2025.05.14
1,"《北京市餐饮服务食品安全 ""夏安"" 专项行动方案》",北京市市场监督管理局,2025.05.14
2,"《""食尚贵州・爱上黔菜"" 餐饮消费券发放方案》",贵州省商务厅,2025.05.16
3,甘肃省人民政府办公厅关于印发甘肃省健康餐饮行动计划(2025—2027年)的通知,甘肃省人民政府,2025.05.20
4,"《""食在广东"" 餐饮消费券发放方案》",广东省商务厅,2025.05.20
...,...,...,...
76,湖北省食品药品安全办、湖北省教育厅、湖北省市场监管局关于印发《湖北省学校食堂复用餐饮具清洗消...,湖北省食品药品安全委员会;湖北省教育厅;湖北省市场监督管理局,2025.10.15
77,江苏省民政厅关于调整江苏省餐饮行业协会2023年度检查结论的通告,江苏省民政厅,2025.10.17
78,《湖南省 2025 年餐饮消费券发放方案》,湖南省商务厅,2025.10.18
79,驻马店市中小学校幼儿园餐饮管理条例,驻马店市人大(含常委会),2025.10.23


In [3]:
import ollama
import pandas as pd
import json
import re
import time
from pathlib import Path
from concurrent.futures import ThreadPoolExecutor, as_completed

# ========================
# 定义打分函数：调用 Qwen 提取 5 个特征
# ========================
def extract_policy_features(row: pd.Series) -> dict:
    """
    输入一行政策数据（含 '标题', '制定机关', '公布日期'）
    返回 dict: {
        "title": str,
        "level": float,
        "novelty": float,
        "fiscal": float,
        "constraint": float,
        "coverage": float,
        "think": str  # 简要推理
    }
    """
    title = str(row.get('标题', '')).strip()
    agency = str(row.get('制定机关', '')).strip()
    date = str(row.get('公布日期', '')).strip()
    
    full_text = f"标题：{title}\n制定机关：{agency}\n公布日期：{date}"

    messages = [
        {
            "role": "system",
            "content": (
                "你是一个政策分析专家，请根据以下政策信息，严格按要求提取5个特征。"
                "你的回答必须是**严格合法的JSON格式**，仅包含以下6个字段："
                ' - "level": 层级（国家=1.0，省=0.6，市=0.3）'
                ' - "novelty": 新颖度（首发=1.3，含修订/调整=1.0，同类型跟发=0.8）'
                ' - "fiscal": 财政性（明确涉资金/税收=1.3，否则=1.0）'
                ' - "constraint": 约束力度（实施细则/规范/整治=1.3，方案/计划=1.0，意见/倡议=0.8）'
                ' - "coverage": 覆盖范围（全行业=1.2，较大范围=1.0，细分环节=0.8）'
                ' - "think": 简要推理（不超过30字）'
                "不要输出任何其他内容、解释、标签或 Markdown。"
            )
        },
        {"role": "user", "content": full_text}
    ]
    try:
        response = ollama.chat(
            model="qwen3:1.7b",  # 可替换为你实际使用的模型名
            messages=messages,
            options={"temperature": 0.1}
        )
        raw = response['message']['content'].strip()

        # 清理可能的 <think> 标签（虽然不应该有）
        cleaned = re.sub(r'(?i)<think>.*?</think>', '', raw, flags=re.DOTALL)
        cleaned = cleaned.strip()

        # 提取 JSON 块
        json_match = re.search(r'\{.*\}', cleaned, re.DOTALL)
        if json_match:
            data = json.loads(json_match.group())
            def safe_float(key, default=1.0):
                try:
                    v = float(data.get(key, default))
                    # 限制合理范围（可选）
                    return v
                except (TypeError, ValueError):
                    return default

            return {
                "title": title,
                "level": safe_float("level"),
                "novelty": safe_float("novelty"),
                "fiscal": safe_float("fiscal"),
                "constraint": safe_float("constraint"),
                "coverage": safe_float("coverage"),
                "think": str(data.get("think", ""))[:50]
            }
        else:
            # 尝试 fallback：逐字段匹配（不推荐，但兜底）
            return {
                "title": title,
                "level": 1.0,
                "novelty": 1.0,
                "fiscal": 1.0,
                "constraint": 1.0,
                "coverage": 1.0,
                "think": "json_parse_failed"
            }

    except Exception as e:
        return {
            "title": title,
            "level": 1.0,
            "novelty": 1.0,
            "fiscal": 1.0,
            "constraint": 1.0,
            "coverage": 1.0,
            "think": f"error:{str(e)[:30]}"
        }

In [4]:
# ========================
# 多线程处理单个 DataFrame
# ========================
def process_policy_sheet(df: pd.DataFrame, max_workers: int = 3) -> pd.DataFrame:
    results = [None] * len(df)

    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        future_to_index = {
            executor.submit(extract_policy_features, row): idx
            for idx, row in df.iterrows()
        }

        for future in as_completed(future_to_index):
            idx = future_to_index[future]
            try:
                res = future.result()
                results[idx] = res
                print(f"[{idx+1}/{len(df)}] Done → Level: {res['level']}, Novelty: {res['novelty']}, Fiscal: {res['fiscal']}, Constraint: {res['constraint']}, Coverage: {res['coverage']}, Think: {res['think']}")
            except Exception as e:
                print(f"Unexpected error at index {idx}: {e}")
                results[idx] = {
                    "title": df.iloc[idx]['标题'],
                    "level": 1.0,
                    "novelty": 1.0,
                    "fiscal": 1.0,
                    "constraint": 1.0,
                    "coverage": 1.0,
                    "think": "crash"
                }
    # 合并结果到原 DataFrame
    df = df.copy()
    df['层级'] = [r['level'] for r in results]
    df['新颖度'] = [r['novelty'] for r in results]
    df['财政性'] = [r['fiscal'] for r in results]
    df['约束力度'] = [r['constraint'] for r in results]
    df['覆盖范围'] = [r['coverage'] for r in results]
    df['分析推理'] = [r['think'] for r in results]

    return df

In [5]:
# ========================
# 主函数：处理 all_sheets 并保存
# ========================
def process_all_sheets(all_sheets: dict, output_file: str, max_workers_per_sheet: int = 3):
    """
    all_sheets: dict of {sheet_name: DataFrame}
    output_file: 输出 Excel 路径，如 'output_政策特征.xlsx'
    """
    output_path = Path(output_file)
    output_path.parent.mkdir(parents=True, exist_ok=True)

    writer = pd.ExcelWriter(output_file, engine='openpyxl')

    for sheet_name, df in all_sheets.items():
        print(f"\n📄 正在处理工作表: {sheet_name} ({len(df)} 条政策)")
        if df.empty:
            print("  ⚠️  空表，跳过")
            df.to_excel(writer, sheet_name=sheet_name, index=False)
            continue

        # 必须包含三列
        required_cols = {'标题', '制定机关', '公布日期'}
        if not required_cols.issubset(df.columns):
            print(f"  ❌ 缺少必要列，跳过。当前列: {list(df.columns)}")
            df.to_excel(writer, sheet_name=sheet_name, index=False)
            continue

        # 处理该 sheet
        df_processed = process_policy_sheet(df, max_workers=max_workers_per_sheet)

        # 写入 Excel
        df_processed.to_excel(writer, sheet_name=sheet_name, index=False)
        print(f"  ✅ 完成: {sheet_name}")

    writer.close()
    print(f"\n🎉 所有工作表处理完成！结果已保存至: {output_file}")

In [ ]:
test_row = pd.Series({
    '标题': '《北京市餐饮服务食品安全"夏安"专项行动方案》',
    '制定机关': '北京市市场监督管理局',
    '公布日期': '2025.05.14'
})
print(extract_policy_features(test_row))
# ' - "level": 层级（国家=1.0，省=0.6，市=0.3）'
# ' - "novelty": 新颖度（首发=1.3，含修订/调整=1.0，同类型跟发=0.8）'
# ' - "fiscal": 财政性（明确涉资金/税收=1.3，否则=1.0）'
# ' - "constraint": 约束力度（实施细则/规范/整治=1.3，方案/计划=1.0，意见/倡议=0.8）'
# ' - "coverage": 覆盖范围（全行业=1.2，较大范围=1.0，细分环节=0.8）'

{'title': '《北京市餐饮服务食品安全"夏安"专项行动方案》', 'level': 1.0, 'novelty': 1.0, 'fiscal': 1.0, 'constraint': 1.3, 'coverage': 1.0, 'think': '北京市市场监管局发布专项方案，覆盖餐饮行业，含修订调整，约束力度强'}


In [7]:
# 3. 调用 Qwen 提取特征并保存
process_all_sheets(
    all_sheets=all_sheets,
    output_file='output/政策特征分析结果.xlsx',
    max_workers_per_sheet=3
)


📄 正在处理工作表: 汽车（国家） (16 条政策)
[3/16] Done → Level: 0.6, Novelty: 1.3, Fiscal: 1.0, Constraint: 0.8, Coverage: 1.2, Think: 行业协会倡议规范行业竞争，无财政资金，覆盖全行业
[1/16] Done → Level: 1.0, Novelty: 1.3, Fiscal: 1.3, Constraint: 1.0, Coverage: 1.2, Think: 三个目录发布，涉及新能源汽车税收和生产管理
[2/16] Done → Level: 1.0, Novelty: 1.3, Fiscal: 1.3, Constraint: 1.3, Coverage: 1.0, Think: 2025年新能源汽车下乡活动通知，工业和信息化部发布，涉及财政补贴，约束力度强，覆盖全行业。
[4/16] Done → Level: 1.0, Novelty: 1.3, Fiscal: 1.0, Constraint: 1.3, Coverage: 1.2, Think: 国家层面首次发布全行业生产一致性检查政策
[5/16] Done → Level: 1.0, Novelty: 1.3, Fiscal: 1.3, Constraint: 1.3, Coverage: 1.2, Think: 工业和信部发布新能源车目录，涵盖全行业，涉及税收与购置税减免
[6/16] Done → Level: 1.0, Novelty: 1.3, Fiscal: 1.0, Constraint: 1.0, Coverage: 1.0, Think: 商务部发布2025年新能源汽车消费季活动，覆盖全国县镇，属首发，无资金税收，属方案计划，约束力度中等
[7/16] Done → Level: 1.0, Novelty: 1.3, Fiscal: 1.3, Constraint: 1.3, Coverage: 1.0, Think: 工信部发布新能源汽车目录，涉及税收与车船税减免，属国家层面，覆盖全行业
[8/16] Done → Level: 1.0, Novelty: 1.3, Fiscal: 1.3, Constraint: 1.0, Coverage: 1.2, Think: 财政部